In [5]:
import sklearn
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import *
from scipy.linalg import sqrtm
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
from sklearn.preprocessing import *
from sklearn.metrics import *
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.neighbors import KNeighborsClassifier
import pickle
from catboost import *
import gc
import catboost
from sklearn.base import clone
from collections import *

In [6]:
class Model():
    
    def __init__(self, main_model, row_model, column_model,
                 count_row_models = 13, count_column_models = 156,
                 main_model_path = None, row_model_path = None, column_model_path = None):
        
        self.main_model    = main_model
        self.row_model     = row_model
        self.column_model  = column_model
        
        self.count_row_models    = count_row_models
        self.count_column_models = count_column_models 

        self.main_model_path     = main_model_path
        self.row_model_path      = row_model_path
        self.column_model_path   = column_model_path
        
        self.init_first_level_models()
        
        if self.main_model_path != None:
            self.main_model  = pickle.load(open(self.main_model_path, "rb"))
        else:
            self.main_model = self.main_model.copy()
            

    def init_first_level_models(self):
        
        if self.row_model_path != None:
            self.row_models = pickle.load(open(self.row_model_path, "rb"))
        else:
            self.row_models = [self.row_model.copy() for i in range(self.count_row_models)]
        
        if self.column_model_path != None:
            self.column_models = pickle.load(open(self.column_model_path, "rb"))
        else:
            self.column_models = [clone(self.column_model) for i in range(self.count_column_models)]
    
    
    def train_row_models(self, data, labels):

        for i in tqdm(range(data.shape[1])):

            train_data = data[:, i, :]

            self.row_models[i].fit(train_data, labels)

        
    def train_column_models(self, data, labels):

        for i in tqdm(range(data.shape[2])):

            train_data = data[:, :, i]

            self.column_models[i].fit(train_data, labels)

    
    def train_main_model(self, data, labels, additional_features = None):
            
        main_preds = self.predict_first_level_models(data)

        if type(additional_features) != type(None):
            main_preds = np.concatenate([main_preds, additional_features], axis = 1)

        self.main_model.fit(main_preds, labels)
        
        return main_preds
    
    
    def predict(self, data, additional_features = None):
        
        main_preds = self.predict_first_level_models(data)
        
        if type(additional_features) != type(None):
            main_preds = np.concatenate([main_preds, additional_features], axis= 1)
        
        preds = self.main_model.predict(main_preds)
        
        return preds
    
    
    def predict_by_column_models(self, data):

        results = np.zeros((data.shape[0], data.shape[2]))

        for i in tqdm(range(data.shape[2])):

            test_data = data[:, :, i]

            preds = self.column_models[i].predict_proba(test_data)[:, 1]

            results[:, i] = preds

        return results

    
    def predict_by_row_models(self, data):

        results = np.zeros((data.shape[0], data.shape[1]))

        for i in tqdm(range(data.shape[1])):

            test_data = data[:, i, :]

            preds = self.row_models[i].predict_proba(test_data)[:, 1]

            results[:, i] = preds

        return results


    def predict_first_level_models(self, data):

        row_preds    = self.predict_by_row_models(data)
        column_preds = self.predict_by_column_models(data)
        
        main_preds = np.concatenate([row_preds, column_preds], axis = 1)

        return main_preds

In [7]:
def predict(model, data, additional_features = None):
        
    main_preds = model.predict_first_level_models(data)

    if type(additional_features) != type(None):
        main_preds = np.concatenate([main_preds, additional_features], axis= 1)

    preds = model.main_model.predict_proba(main_preds)

    return preds[:, 1]

In [8]:
with open("../input/models/abs_model", "rb") as file:
    model = pickle.load(file)

In [9]:
def get_stats(data):
    data_group = data.groupby(data.index)

    data_max  = data_group.max()
    data_min  = data_group.min()
    data_mean = data_group.mean()

    data_stat_features = pd.concat([data_max, data_min, data_mean], axis = 1).to_numpy(np.float16)

    del data_max
    del data_min
    del data_mean
    
    return data_stat_features

In [10]:
first = True

for i in tqdm(range(1, 4)):
    data = pd.read_parquet("../input/amex-test-repeat/test{}.parquet".format(i)).astype(np.float16)
    
    cat_features  = ['B_30', 'B_38', 'D_114', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64',  'D_68']
    
    indexes = data.index.drop_duplicates()
    data    = data.drop(cat_features, axis=1)
    
    data_stat_features = get_stats(data)
    
    data = data.to_numpy(np.float16)
    data = data.reshape(-1, 13, data.shape[1])
    
    preds = predict(model, data, data_stat_features)
    
    preds = pd.DataFrame({"customer_ID" : indexes, "prediction" : preds}).set_index("customer_ID")    
    
    if first:
        preds.to_csv("submit.csv")
        first = False
    else:
        preds.to_csv("submit.csv", mode = "a", header = False)
        
    del data
    del preds
    del data_stat_features
    
    gc.collect()

100%|██████████| 13/13 [02:51<00:00, 13.20s/it]

100%|██████████| 13/13 [02:53<00:00, 13.37s/it]

100%|██████████| 13/13 [05:07<00:00, 23.62s/it]

100%|██████████| 3/3 [17:40<00:00, 353.46s/it]


In [11]:
preds = pd.read_csv("./submit.csv")
preds = preds.drop(488255, axis=0)

In [19]:
preds = preds.set_index("customer_ID")

In [20]:
preds.to_csv("submit3.csv")

In [21]:
preds

,prediction
customer_ID,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.026836
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.008526
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.025309
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.440981
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.805947
...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.031464
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.858967
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.651606
